<a href="https://colab.research.google.com/github/Blaydor09/AnalizadorEnriquecedorL-xico/blob/main/enriquecedorLexico_DC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Analizador y Enriquecedor Léxico para Español

Este notebook permite **analizar automáticamente palabras en español** y alimentar un archivo léxico estructurado en formato JSON, compatible con analizadores sintácticos.

### 🚀 ¿Qué hace este notebook?
- Procesa una **lista de palabras** o un archivo `.txt` con nuevas palabras.
- Descarga automáticamente una lista con las **50,000 palabras más frecuentes en español** desde una fuente pública.
- Elimina datos innecesarios como frecuencias de uso y conserva solo las palabras.
- Usa **spaCy** con un modelo entrenado para español para identificar:
  - Tipo gramatical (verbo, sustantivo, determinante, etc.)
  - Género (masculino, femenino, neutral)
  - Número (singular, plural)
  - Persona, tiempo, modo, etc. (cuando corresponda)
- Clasifica **adverbios** y **preposiciones** según significado usando reglas personalizadas.
- Reconoce **nombres propios** y los etiqueta con género o categoría (`país`, `ciudad`) según el caso.
- Actualiza automáticamente el archivo principal `lexico.json`.
- Genera un nuevo archivo `lexico_extendido.json` con solo las palabras nuevas añadidas.

### 🧠 ¿Para quién es este notebook?
Ideal para desarrolladores de:
- Analizadores sintácticos en español
- Chatbots y asistentes de lenguaje natural
- Sistemas educativos de gramática
- Aplicaciones lingüísticas o de procesamiento de texto

---

✅ **Autor:** *Jose Fernando Nacif* — Proyecto: *Diseño de Compiladores*  
📅 **Última actualización:** *31/05/2025*


## === OJO ===
### Hay que subir el archivo lexico que se esta usando en el compilador con los datos actuales...
### En el caso de no tener el archivo usar la estructura de json de la ultima celda de este notebook. Crear el archivo lexico.json

In [ ]:
!pip install -U spacy
!python -m spacy download es_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 45.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 69.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 26.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: thinc
    Found existing installation: thinc 8.2.5
    Uninstalling thinc-8.2.5:
      Successfully uninstalled thinc-8.2.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.7.5
    Uninstalling spacy-3.7.5:
      Successfully uninstalled spacy-3.7.5
ERROR: pip's

> Instalación de spaCy y modelo en español

In [ ]:
import requests
import os

url = "https://raw.githubusercontent.com/hermitdave/FrequencyWords/master/content/2016/es/es_50k.txt"
archivo_txt = "es_50k.txt"

if not os.path.exists(archivo_txt):
    r = requests.get(url)
    with open(archivo_txt, "wb") as f:
        f.write(r.content)
    print(f"✅ Archivo descargado: {archivo_txt}")
else:
    print(f"📄 Ya existe el archivo: {archivo_txt}")


✅ Archivo descargado: es_50k.txt


> Descargar archivo de 50K palabras

In [ ]:
def limpiar_frecuencias(path_original, path_limpio="es_50k_limpio.txt"):
    palabras = []
    with open(path_original, "r", encoding="utf-8") as f_in:
        for linea in f_in:
            palabra = linea.strip().split(" ")[0]
            if palabra: palabras.append(palabra)

    with open(path_limpio, "w", encoding="utf-8") as f_out:
        f_out.write("\n".join(palabras))

    print(f"✅ Archivo limpio guardado como: {path_limpio}")
    return palabras

lista_limpia = limpiar_frecuencias("es_50k.txt")


✅ Archivo limpio guardado como: es_50k_limpio.txt


> Limpiar frecuencias y generar archivo limpio

In [ ]:
import spacy
import json

nlp = spacy.load("es_core_news_md")

normalizar_valores = {
    "Masc": "masculino", "Fem": "femenino",
    "Sing": "singular", "Plur": "plural",
    "1": "primera", "2": "segunda", "3": "tercera",
    "Inf": "infinitivo", "Fin": "conjugado",
    "empty": "neutral",
    "Pres": "presente", "pres": "presente",
    "Past": "pasado", "past": "pasado",
    "Fut": "futuro", "fut": "futuro",
    "Ind": "indicativo", "ind": "indicativo",
    "Subj": "subjuntivo", "sub": "subjuntivo", "subj": "subjuntivo",
    "Imp": "imperativo", "imp": "imperativo"
}



def normalizar(valor):
    if isinstance(valor, list) and valor:
        valor = valor[0]
    return normalizar_valores.get(valor, valor.lower() if isinstance(valor, str) else valor)

CLASIFICACION_ADVERBIOS = {
    "ayer": "tiempo", "hoy": "tiempo", "mañana": "tiempo", "temprano": "tiempo",
    "siempre": "frecuencia", "nunca": "frecuencia", "aquí": "lugar", "allí": "lugar",
    "cerca": "lugar", "lejos": "lugar", "rápidamente": "modo", "lentamente": "modo",
    "bien": "modo", "mal": "modo"
}

CLASIFICACION_PREPOSICIONES = {
    "a": "PREPOSICION_A", "de": "PREPOSICION_DE", "por": "PREPOSICION_POR",
    "para": "PREPOSICION_PARA", "en": "PREPOSICION_EN", "con": "PREPOSICION_CON",
    "sin": "PREPOSICION_SIN", "sobre": "PREPOSICION_SOBRE", "bajo": "PREPOSICION_BAJO"
}

CLASIFICACION_NOMBRES_PROPIOS = {
    "argentina": "país", "bolivia": "país", "chile": "país", "méxico": "país",
    "españa": "país", "francia": "país", "la paz": "ciudad", "santa cruz": "ciudad",
    "parís": "ciudad", "madrid": "ciudad"
}


> Inicializar spaCy y definiciones gramaticales

In [ ]:
def clasificar_token(token):
    tag = token.pos_
    morph = token.morph
    info = {}

    if tag == "DET":
        categoria = "determinantes"
        info = {
            "tipo": "DETERMINANTE",
            "genero": normalizar(morph.get("Gender")) if morph.get("Gender") else "neutral",
            "numero": normalizar(morph.get("Number")) if morph.get("Number") else "singular"
        }

    elif tag == "NOUN":
        categoria = "sustantivos"
        info = {
            "tipo": "SUSTANTIVO",
            "genero": normalizar(morph.get("Gender")) if morph.get("Gender") else "neutral",
            "numero": normalizar(morph.get("Number")) if morph.get("Number") else "singular"
        }

    elif tag in ("VERB", "AUX"):
      categoria = "verbos"
      if morph.get("VerbForm") == ["Inf"]:
          info = {
              "tipo": "VERBO",
              "tiempo": "infinitivo"
          }
      else:
          info = {
              "tipo": "VERBO",
              "tiempo": normalizar(morph.get("Tense")) if morph.get("Tense") else "indefinido",
              "modo": normalizar(morph.get("Mood")) if morph.get("Mood") else "indefinido",
              "persona": normalizar(morph.get("Person")) if morph.get("Person") else "indefinido",
              "numero": normalizar(morph.get("Number")) if morph.get("Number") else "indefinido"
          }

    elif tag == "ADJ":
        categoria = "adjetivos"
        info = {
            "tipo": "ADJETIVO",
            "genero": normalizar(morph.get("Gender")) if morph.get("Gender") else "neutral",
            "numero": normalizar(morph.get("Number")) if morph.get("Number") else "singular"
        }

    elif tag == "PRON":
        categoria = "pronombres"
        info = {
            "tipo": "PRONOMBRE",
            "persona": normalizar(morph.get("Person")) if morph.get("Person") else "tercera",
            "numero": normalizar(morph.get("Number")) if morph.get("Number") else "singular"
        }
        if morph.get("Gender"):
            info["genero"] = normalizar(morph.get("Gender"))

    elif tag == "PROPN":
        categoria = "nombres_propios"
        texto_normalizado = token.text.lower()
        info = { "tipo": "NOMBRE_PROPIO" }
        if texto_normalizado in CLASIFICACION_NOMBRES_PROPIOS:
            info["categoria"] = CLASIFICACION_NOMBRES_PROPIOS[texto_normalizado]
        elif morph.get("Gender"):
            info["genero"] = normalizar(morph.get("Gender"))

    elif tag == "ADV":
        categoria = "adverbios"
        clase = CLASIFICACION_ADVERBIOS.get(token.text.lower(), "modo")
        info = {
            "tipo": "ADVERBIO",
            "clase": clase
        }

    elif tag == "ADP":
        categoria = "preposiciones"
        tipo = CLASIFICACION_PREPOSICIONES.get(token.text.lower(), f"PREPOSICION_{token.text.upper()}")
        info = {
            "tipo": tipo
        }

    else:
        return None, None

    return categoria, info


> Clasificación gramatical de palabras

In [ ]:
def procesar_batch(palabras, lexico, palabras_existentes, nuevas_palabras):
    verbos_actualizados = []

    for palabra in palabras:
        doc = nlp(palabra)
        for token in doc:
            categoria, info = clasificar_token(token)
            if not categoria:
                continue

            palabra_lower = palabra.lower()

            if categoria == "verbos":
                ya_existe = palabra_lower in palabras_existentes.get(categoria, set())

                lexico["palabras"].setdefault(categoria, {})[palabra_lower] = info
                nuevas_palabras.setdefault(categoria, {})[palabra_lower] = info

                if ya_existe:
                    print(f"🔁 Verbo existente actualizado: '{palabra_lower}'")
                    verbos_actualizados.append(palabra_lower)

            else:
                if palabra_lower not in palabras_existentes.get(categoria, set()):
                    lexico["palabras"].setdefault(categoria, {})[palabra_lower] = info
                    nuevas_palabras.setdefault(categoria, {})[palabra_lower] = info

    return verbos_actualizados


# === Leer y preparar lexico.json ===
with open("lexico.json", "r", encoding="utf-8") as f:
    lexico = json.load(f)

palabras_existentes = {cat: set(lexico["palabras"].get(cat, {}).keys()) for cat in lexico["palabras"]}
nuevas_palabras = {}

# === Procesar lote de palabras ===
verbos_actualizados = procesar_batch(lista_limpia[:5000], lexico, palabras_existentes, nuevas_palabras)

# === Guardar archivos ===
with open("lexico.json", "w", encoding="utf-8") as f:
    json.dump(lexico, f, indent=4, ensure_ascii=False)

with open("lexico_extendido.json", "w", encoding="utf-8") as f:
    json.dump({"palabras": nuevas_palabras}, f, indent=4, ensure_ascii=False)

# === Resumen final ===
total_nuevas = sum(len(v) for v in nuevas_palabras.values())
print(f"\n✅ {total_nuevas} palabras nuevas agregadas en {len(nuevas_palabras)} categorías.")
print(f"🔁 {len(verbos_actualizados)} verbos existentes fueron actualizados.")



✅ 4924 palabras nuevas agregadas en 8 categorías.
🔁 0 verbos existentes fueron actualizados.


> Procesamiento en lote + Guardado

In [ ]:
"""
{
    "palabras": {
        "determinantes": {

        },
        "sustantivos": {

        },
        "verbos": {

        },
        "adjetivos": {

        },
        "pronombres": {

        },
        "nombres_propios": {

        },
        "adverbios": {

        },
        "preposiciones": {

        }
    }
}
"""